In [1]:
import scqubits as sc
import qutip as qt
import numpy as np
from matplotlib import pyplot as plt
import array_to_latex as a2l
from scipy.linalg import inv

In [2]:
omega = 6.67
g1 = .2

qubit = sc.Transmon(
    EJ = 16.0, 
    EC = 0.2, 
    ng = 0, 
    ncut = 30,
    truncated_dim=10
)

res_a = sc.Oscillator(E_osc=omega, truncated_dim = 20)
res_b = sc.Oscillator(E_osc=omega, truncated_dim = 20)

hs = sc.HilbertSpace([qubit, res_a, res_b])

hs.add_interaction(
    g_strength = g1,
    op1 = qubit.n_operator,
    op2 = res_a.annihilation_operator,
    add_hc = True
)

hs.generate_lookup()

In [3]:
tot_trunc = 40
def trunc(op): #method for truncating Qobjs
    return qt.Qobj(op[:tot_trunc, :tot_trunc])

In [4]:
a = hs.op_in_dressed_eigenbasis(op=res_a.annihilation_operator) #convert subsystem operators to dressed eigenbasis rep
b = hs.op_in_dressed_eigenbasis(op=res_b.annihilation_operator)

In [5]:
#subsystem Hamiltonians
res_a_ham = omega*a.dag()*a
res_b_ham = omega*b.dag()*b
qubit_ham = hs.op_in_dressed_eigenbasis(op=qubit.hamiltonian)

#bare Hamiltonian
bare_ham = trunc(qubit_ham + res_a_ham + res_b_ham)

def state_transformation(state, t): #move to interaction picture
    return (2j*np.pi*bare_ham*t).expm()*state

In [6]:
#calculate dispersive shift
w2 = hs.energy_by_bare_index((1,1,0)) - hs.energy_by_bare_index((1,0,0))
w1 = hs.energy_by_bare_index((0,1,0)) - hs.energy_by_bare_index((0,0,0))
chi = (w2-w1)/2
g = -np.sqrt(3)/2 * chi

In [7]:
g/(2*np.pi)

0.0006293699283061227

In [8]:
bsc = trunc(g/2*(a*b.dag()+b*a.dag())) #beamsplitter Hamiltonian

In [9]:
(evals,) = hs["evals"]
diag_dressed = (
    2*np.pi*qt.Qobj(np.diag(evals),
    dims = [hs.subsystem_dims] * 2)
)

In [10]:
diag_dressed_trunc = trunc(diag_dressed)

In [11]:
def conditional_basis(qubit):
    return [hs.dressed_index((qubit, a,b)) for (a,b) in [(0,0),(0,1),(1,0),(1,1)]]

In [12]:
def prop_col(initial, qubit):
    col = [] 
    tlist = np.linspace(0, 2*np.pi/g, 1000)
    result = qt.sesolve(
            diag_dressed_trunc + 2*np.pi*bsc,
            qt.basis(tot_trunc, initial),
            tlist,
        )
    for idx in conditional_basis(qubit):
        col.append(state_transformation(result.states[-1], tlist[-1])[idx][0][0])
    return col

In [13]:
def conditional_propagator(qubit):
    prop = []
    for idx in conditional_basis(qubit):
        prop.append(prop_col(idx, qubit))
    return prop

In [14]:
Ug = conditional_propagator(0)

In [15]:
Uf = conditional_propagator(2)

In [16]:
a2l.to_ltx(inv(Ug)*Uf)

\begin{bmatrix}
 -0.10 + 0.98j &  0.00 + 0.00j &  0.00 + 0.00j &  0.00 + 0.00j\\
  0.00 + 0.00j &  0.63 + 0.65j & -0.00 + -0.02j &  0.00 + 0.00j\\
  0.00 + 0.00j &  0.02 + 0.00j & -0.62 + -0.65j &  0.00 + 0.00j\\
  0.00 + 0.00j &  0.00 + 0.00j &  0.00 + 0.00j & -0.50 + -0.54j
\end{bmatrix}


In [17]:
a2l.to_ltx(np.matrix(Uf))

\begin{bmatrix}
  0.90 + -0.42j &  0.00 + 0.00j &  0.00 + 0.00j & -0.00 + -0.00j\\
  0.00 + 0.00j &  0.10 + -0.90j & -0.27 + -0.09j &  0.00 + 0.00j\\
  0.00 + 0.00j &  0.16 + -0.25j &  0.61 + 0.61j &  0.00 + 0.00j\\
 -0.00 + 0.00j &  0.00 + 0.00j &  0.00 + 0.00j & -0.33 + 0.63j
\end{bmatrix}


In [18]:
a2l.to_ltx(np.matrix(Ug))

\begin{bmatrix}
 -0.52 + -0.86j &  0.00 + 0.00j &  0.00 + 0.00j & -0.00 + 0.00j\\
  0.00 + 0.00j & -0.64 + -0.77j &  0.03 + -0.06j &  0.00 + 0.00j\\
  0.00 + 0.00j &  0.03 + -0.06j & -0.96 + 0.02j &  0.00 + 0.00j\\
 -0.00 + -0.00j &  0.00 + 0.00j &  0.00 + 0.00j & -0.33 + -0.90j
\end{bmatrix}
